### Desenvolvimento e Entrega do 3º Checkpoint - Data Science e Statistical
 
**Nomes + RM dos integrantes:**
- Guilherme Akio - 98582
- Fabrício Saavedra - 97631
 
**Turma:** 2ESPW
 
**Ano:** 2024

> Link do repositório: https://github.com/Fabr1c100/GS_2SEM_DataScience.git

___ 
### Descrição do Projeto
.

> Base: https://www.kaggle.com/datasets/jainaru/global-ecological-footprint-2023/data
